In [1]:
import pandas as pd
import time
import os
import cv2
import tensorflow as tf
from text_detection.text_detection import text_detection, load_text_model
from text_detection.process import *
from text_detection.regex import *
from text_detection.nutrient_list import *
from text_detection.spacial_map import *
from PIL import Image
import numpy as np
from crop import ImageCropper

In [2]:
def crop(image_obj, coords, saved_location, extend_ratio=0, SAVE=False):
    nx = image_obj.shape[1]
    ny = image_obj.shape[0]

    modified_coords = (
        int(coords[0] - extend_ratio * nx), 
        int(coords[1] - extend_ratio * ny), 
        int(coords[2] + extend_ratio * nx), 
        int(coords[3] + extend_ratio * ny)
    )
    cropped_image = image_obj[modified_coords[1]:modified_coords[3], modified_coords[0]:modified_coords[2]]

    if SAVE:
        cv2.imwrite(saved_location, cropped_image)

    return cropped_image

In [3]:
class IoULoss(tf.keras.losses.Loss):
    def call(self, y_true, y_pred):
        intersection = tf.reduce_sum(tf.minimum(y_true, y_pred), axis=-1)
        union = tf.reduce_sum(tf.maximum(y_true, y_pred), axis=-1)
        iou = intersection / union
        return 1 - iou

In [4]:
def detect(cropped_image, debug=False):
    start_time = time.time() 

    cropped_image_cv = cv2.cvtColor(np.array(cropped_image), cv2.COLOR_RGB2BGR)

    cropped_image_cv = preprocess_for_ocr(cropped_image_cv, 3)

    if debug:
        debug_folder = 'C:/Users/koust_fiminqv/Desktop/thesis/debug'
        os.makedirs(debug_folder, exist_ok=True)
        cv2.imwrite(os.path.join(debug_folder, 'output-opt.png'), cropped_image_cv)


    text_blob_list = text_detection(cropped_image_cv)

    if debug:
        time_taken = time.time() - start_time
        print("Time Taken to detect bounding boxes for text: %.5fs" % time_taken)

    text_location_list = []  
    nutrient_dict = {} 

    counter = 0
    for blob_cord in text_blob_list:
        if debug:
            counter += 1
            word_image = crop(cropped_image_cv, blob_cord, os.path.join(debug_folder, "{}.jpg".format(counter)), 0.005, True)
        else:
            word_image = crop(cropped_image_cv, blob_cord, "", 0.005, False)

        if word_image.size == 0:
            continue

        word_image = preprocess_for_ocr(word_image)
        if (word_image.shape[1] > 0 and word_image.shape[0] > 0):
            text = ocr(word_image, 1, 7)

            if debug:
                print(text)

            if text:
                center_x = (blob_cord[0] + blob_cord[2]) / 2
                center_y = (blob_cord[1] + blob_cord[3]) / 2
                box_center = (center_x, center_y)

                new_location = {
                    'bbox': blob_cord,
                    'text': text,
                    'box_center': box_center,
                    'string_type': string_type(text)
                }
                text_location_list.append(new_location)
    for text_dict in text_location_list:
        if text_dict['string_type'] == 2:
            for text_dict_test in text_location_list:
                if position_definer(text_dict['box_center'][1], text_dict_test['bbox'][1], text_dict_test['bbox'][3]) and text_dict_test['string_type'] == 1:
                    text_dict['text'] += ' ' + text_dict_test['text']
                    text_dict['string_type'] = 0

    fuzdict = make_fuzdict('data/nutrients.txt')
    for text_dict in text_location_list:
        if text_dict['string_type'] == 0:
            text = clean_string(text_dict['text'])
            if fuz_check_for_label(text, fuzdict, debug):
                label_name, label_value = get_fuz_label_from_string(text, fuzdict, debug)
                nutrient_dict[label_name] = separate_unit(label_value)

    if debug:
        time_taken = time.time() - start_time
        print("Total Time Taken: %.5fs" % time_taken)

    return nutrient_dict

In [5]:
def run_detection(image_path, cropper_model_path, debug=False):
    cropper_model = tf.keras.models.load_model(cropper_model_path, custom_objects={'IoULoss': IoULoss})
    image_cropper = ImageCropper(model=cropper_model)
    cropped_image = image_cropper.process_image(image_path)
    if debug:
        debug_folder = 'C:/Users/koust_fiminqv/Desktop/thesis/debug'
        os.makedirs(debug_folder, exist_ok=True)
        cropped_image_path = os.path.join(debug_folder, 'cropped_image.jpg')
        cropped_image.save(cropped_image_path)

    load_text_model()
    nutrient_dict = detect(cropped_image, debug)
    return nutrient_dict

In [6]:
image_path = 'C:/Users/koust_fiminqv/Desktop/thesis/test_image/test3.jpg'
cropper_model_path = 'C:/Users/koust_fiminqv/Desktop/thesis/models/resnet_tuned_model.h5'
debug = False  

nutrient_dict = run_detection(image_path, cropper_model_path, debug)
df = pd.DataFrame(list(nutrient_dict.items()), columns=['Nutrient', 'Value'])
print(df)

1/1 [==============================] - 1s 1s/step
Instructions for updating:
Use tf.gfile.GFile.
Text Weights Loaded!
        Nutrient      Value
0           Salt  (16.0, g)
1        Protein   (2.7, g)
2      Total Fat  (16.9, g)
3         Sugars  (12.6, g)
4  Saturated Fat   (3.3, g)
